# IMDB 50K Movie Reviews - Sentiment Classification with MLP

## Overview

### In the next exercises, you will work with the IMDB 50K Movie Reviews dataset to build a sentiment analysis model using a Multi-Layer Perceptron (MLP). You will practice essential steps in the data science pipeline such as data loading, preprocessing, feature generation, and training/testing a neural network model. This exercise should be completed using the `pandas`, `nltk`, `sklearn`, and `torch` libraries.


### Exercise 1: Data Loading and Exploration
**Objective**: Load the IMDB dataset and explore its structure.

1. **Load the dataset** using `pandas`. The dataset is in the `data/` folder and the file name is `imdb_dataset.zip`. **Hint: you can load zip files with pandas by passing `compression='zip'` tp `pd.read_csv`**
2. **Explore the dataset** by checking for missing values and getting a summary of the data. 
    - Check the shape of the dataset.
    - Get the distribution of the sentiment labels (positive/negative reviews).
3. Print the first few reviews and their corresponding labels.


In [1]:
import modin.pandas as pd
df = pd.read_csv("../../data/imdb_dataset.zip", compression='zip')
df

2024-10-04 10:01:07,237	INFO worker.py:1786 -- Started a local Ray instance.


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [2]:
df.shape

(50000, 2)

In [3]:
df['sentiment'].value_counts()

the groupby keys will be sorted anyway, although the 'sort=False' was passed. See the following issue for more details: https://github.com/modin-project/modin/issues/3571.


sentiment
negative    25000
positive    25000
Name: count, dtype: int64

### Exercise 2: Splitting the Data
**Objective**: Split the data into training, validation and test sets.

1. Split the dataset into features (reviews) and labels (sentiment).
2. Use `train_test_split` from `sklearn` to split the dataset into training, validation and test sets (use an 60/20/20 split).
3. Print the sizes of the training and test sets to ensure the splits were done correctly.

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, stratify=df['sentiment'], random_state=42)
train, valid = train_test_split(train,  test_size=0.25, stratify=train['sentiment'], random_state=42)

train.shape, test.shape,  valid.shape

((30000, 2), (10000, 2), (10000, 2))

### Exercise 3: Text Preprocessing
**Objective**: Preprocess the text data to prepare it for feature generation.

1. Lowercase the text data. **Hint: python has a built-in string method for this**.
2. Remove any URL from the reviews. **Hint: you can use regular expressions for this**.
3. Remove non-word and non-whitespace characters (punctuation, special characters, etc.). **Hint: you can use regular expressions for this**.
4. Remove digits. **Hint: you can use regular expressions for this**.
5. Tokenize the reviews into individual words. **Hint: you can use the `nltk` library for this**.
6. Remove stopwords. **Hint: you can use the `nltk` library for this**.
7. Perform stemming or lemmatization. **Hint: you can use the `nltk` library for this**.
8. Apply the preprocessing steps to both the training, validation and test sets.

In [8]:
# Lowercase
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


nltk.download('stopwords')
nltk.download('punkt')
stemer =  PorterStemmer()

def preprocess(text: str) -> str:
    text = text.to_lower()
    link_pattern = r'https\S+|www\S'
    text =  re.sub(link_pattern, '', text)
    non_alphanumeric_pattern = r'\W'
    text =  re.sub(non_alphanumeric_pattern, ' ', text)
    digt_pattern = r'\d'
    text = re.sub(digt_pattern, '', text)
    tokenize = nltk.word_tokenize(text)
    words = [word for word in tokenize if word not in stopwords.words('english')]
    stems = [stemer.stem(word) for word in words]

    return " ".join(stems)

x_train, y_train = train['review'], train['sentiment']
x_valid, y_valid = valid['review'], valid['sentiment']
x_test, y_test = test['review'], test['sentiment']

x_train =  x_train.apply(preprocess)
x_valid = x_valid.apply(preprocess)
x_test = x_test.apply(preprocess)

x_train
# train['review'] = train['review'].apply(preprocess())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diogo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diogo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


RayTaskError(RaySystemError): [36mray::remote_exec_func()[39m (pid=14776, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1806, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1840, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 943, in ray._raylet.raise_if_dependency_failed
ray.exceptions.RayTaskError: [36mray::_remote_exec_single_chain()[39m (pid=23848, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1806, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1840, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 943, in ray._raylet.raise_if_dependency_failed
ray.exceptions.RaySystemError: System error: maximum recursion depth exceeded
traceback: Traceback (most recent call last):
  File "c:\Users\diogo\miniconda3\envs\deep-learning\Lib\site-packages\ray\_private\serialization.py", line 423, in deserialize_objects
    obj = self._deserialize_object(data, metadata, object_ref)
          ^^^^^^^^^^^^^^^^^^^^^

### Exercise 4: Feature Generation (TF-IDF)
**Objective**: Convert the preprocessed text data into numerical features using TF-IDF.

1. Use the `TfidfVectorizer` from `sklearn` to convert the reviews into numerical vectors.
2. Limit the maximum number of features to 5,000 to reduce the dimensionality.
3. Fit the vectorizer on the training set and transform both the training and test sets.
4. Print the shape of the transformed feature sets to confirm the conversion.

### Exercise 5: Building the MLP Model (PyTorch)
**Objective**: Build a simple Multi-Layer Perceptron (MLP) for binary classification.

1. Define the MLP model using `torch.nn.Module`. The model should have:
    - An input layer that matches the size of the TF-IDF features.
    - Two hidden layers with ReLU activations.
    - A single output layer with a sigmoid activation function.
2. Print the model summary.


### Exercise 6: Training the Model
**Objective**: Train the MLP model on the training data.

1. Convert the TF-IDF feature matrices and labels into PyTorch tensors (the label needs to be binarized).
2. Define the loss function (`BCELoss` for binary classification) and the optimizer (`Adam`).
3. Implement a training loop to train the model for a specified number of epochs (e.g., 50).
4. Monitor the training and validation loss during training.

### Exercise 7: Model Evaluation
**Objective**: Evaluate the performance of the trained model on the test data.

1. Use the trained model to make predictions on the test set.
2. Calculate the accuracy of the model on the test data.
3. Print the test accuracy.

### **Exercise 8: Saving the Trained Model**

1. Save the model's state_dict using `torch.save()`. 

2. Save the entire model, including its architecture and weights.

3. Demonstrate how to load the saved model and use it for making predictions on new data.
